# Set up

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm


import os
import pickle

import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# Data

In [ ]:
# Y_RealCase_train = pd.read_csv('Data/Y_train_std.csv', header=None, delimiter=',')
# df_new = Y_RealCase_train.drop(Y_RealCase_train.columns[-18:], axis=1)
# Y_data_train = df_new.drop(df_new.columns[17], axis=1)

In [ ]:
# np.savetxt("Y_RealCase_train.csv", Y_data_train.values, delimiter=",", fmt="%.8f")

In [2]:
X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values

Y_train_std = pd.read_csv('RealCase/Y_RealCase_train.csv', header=None, delimiter=',').values
Y_test_std = pd.read_csv('RealCase/Y_RealCase.csv', header=None, delimiter=',').values

train_x = torch.tensor(X_train, dtype=torch.float32)

train_y = torch.tensor(Y_train_std, dtype=torch.float32)
test_y = torch.tensor(Y_test_std, dtype=torch.float32)


torch.set_default_dtype(torch.float32)

# Emulator

In [3]:
Device = 'cpu'

## MVGP

In [4]:
num_latents = 32
num_inducing = 400
covar_type = 'RBF'

In [6]:
MVGP_models, MVGP_likelihoods = Training.train_MultitaskVGP_minibatch(
                train_x=train_x.to(Device),
                train_y=train_y.to(Device),
                covar_type=covar_type,
                num_latents=num_latents,
                num_inducing=num_inducing,
                lr_hyper=0.01,
                lr_variational=0.1,
                num_iterations=10000,
                patience=10,
                device=Device,
                batch_size=512,
                eval_every=100,
                eval_batch_size=1024
            )

Training: 100%|█████████▉| 9999/10000 [1:08:59<00:00,  2.42it/s, full_loss=-64.2]  


In [7]:
checkpoint = {
    'model_state_dict': MVGP_models.state_dict(),
    'likelihood_state_dict': MVGP_likelihoods.state_dict(),
}

torch.save(checkpoint, 'mvgp_checkpoint_RealCase.pth')

# PE

In [11]:
input_point = test_y.view(-1)

local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k=100)

bounds = bound.get_bounds(local_train_x)

In [12]:
input_point

tensor([ 3.5505, -1.4240, -2.4485, -1.4250, -3.0101, -3.5393,  0.0193, -2.2150,
        -1.1616, -0.3620, -0.3273, -1.9095,  0.8713, -1.6603,  1.2339,  0.0555,
         0.2223,  0.2350,  5.1988, 11.3034,  6.4942,  3.6123,  1.6795,  2.3737,
         1.6453,  1.5493,  0.6599,  2.0654,  1.0090,  0.9459,  2.3916,  2.6009,
         1.6791])

In [23]:
row_idx = 0

estimated_params_tmp, Loss_tmp = Estimation.multi_start_estimation(
        MVGP_models, MVGP_likelihoods, row_idx, test_y, bounds,
        Estimation.estimate_params_for_one_model_Adam, num_starts=16, num_iterations=2000, lr=0.01,
        patience=15, attraction_threshold=0.1, repulsion_strength=0.1, device=Device
    )

In [24]:
Loss_tmp

10.169404029846191

In [27]:
train_x

tensor([[4.0368, 1.4703, 4.7201,  ..., 3.5262, 0.5741, 1.6669],
        [1.2786, 4.2496, 1.6591,  ..., 0.1037, 2.8112, 4.3165],
        [1.8247, 0.9324, 3.2145,  ..., 1.4873, 3.8580, 2.8133],
        ...,
        [4.9321, 2.0119, 4.4979,  ..., 1.1641, 0.6235, 4.6644],
        [2.9204, 4.0962, 1.6257,  ..., 1.4481, 1.4271, 2.6840],
        [2.2186, 0.4472, 3.1952,  ..., 3.9552, 4.6432, 0.9042]])

In [29]:
train_pre = Prediction.preds_for_one_model(MVGP_models, MVGP_likelihoods, train_x)

In [36]:
torch.mean((train_pre.view(4539, 33) - train_y) ** 2)

tensor(0.0005, grad_fn=<MeanBackward0>)

In [33]:
train_y

tensor([[ 1.0668,  0.7744,  0.8051,  ...,  1.4691,  1.5461,  1.3133],
        [ 0.8924, -4.6733, -4.6431,  ..., -0.7121, -0.7879, -0.4826],
        [-0.9402,  0.6595,  0.6192,  ..., -0.8106, -0.8102, -0.8342],
        ...,
        [-0.1767, -1.9733, -1.8386,  ...,  0.0343,  0.1716,  0.3470],
        [-0.5664,  0.1074,  0.2103,  ...,  0.0139,  0.0479,  0.1916],
        [ 0.8573,  0.1481,  0.1295,  ..., -0.3966, -0.5752, -0.7744]])

In [34]:
train_pre

tensor([ 1.0426,  0.7895,  0.8131,  ..., -0.3996, -0.5785, -0.7712],
       grad_fn=<ViewBackward0>)

In [25]:
mcmc_result_Uniform = Estimation.run_mcmc_Uniform(
        Prediction.preds_distribution, MVGP_models, MVGP_likelihoods, 
        row_idx, test_y, bounds, 
        num_sampling=1200, warmup_step=300, num_chains=1, device=Device
    )

Warmup:   1%|          | 13/1500 [01:14, 19.23s/it, step size=1.41e-06, acc. prob=0.498]

KeyboardInterrupt: 